In [16]:
from Crypto.Hash import keccak

In [18]:
# Pcurve = 115792089237316195423570985008687907853269984665640564039457584007908834671663
N = 115792089237316195423570985008687907852837564279074904382605163141518161494337
# Ac = 0
# Bc = 7
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424

In [20]:
def egcd(a, b):
     if a == 0:
          return (b, 0, 1)
     else:
          g, y, x = egcd(b % a, a)
          return (g, x - (b // a) * y, y)
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exemption('modular inverse does not exist')
    else:
        return x % m

In [21]:
from fastecdsa.curve import Curve
from fastecdsa.point import Point
from fastecdsa.curve import secp256k1

In [24]:
msgHash = '0x' + keccak.new(data=b'This is the right message', digest_bits=256).hexdigest()
msgHash

'0xad96fdb825e1cf5f41c712b42a0af7faa11ed7e6e2b8d37bfa10fc9df40adb27'

In [25]:
GenPoint = Point(Gx, Gy, curve=secp256k1)
GenPoint

X: 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
Y: 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
(On curve <secp256k1>)

In [39]:
sk = 1
vk = sk * GenPoint
concat_x_y = vk.x.to_bytes(32, byteorder='big') + vk.y.to_bytes(32, byteorder='big')
eth_addr = '0x' + keccak.new(data=concat_x_y,digest_bits=256).hexdigest()[-40:]
vk,eth_addr

(X: 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
 Y: 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
 (On curve <secp256k1>),
 '0x7e5f4552091a69125d5dfcb7b8c2659029395bdf')

In [79]:
# k - random number
k = 57896044618658097711785492504343953926418782139537452191302581570759080747168 
XY1 = k * GenPoint
r = XY1.x % N
r,hex(r)

(86918276961810349294276103416548851884759982251107,
 '0x3b78ce563f89a0ed9414f5aa28ad0d96d6795f9c63')

In [87]:
s = (r*sk +int(msgHash,16) * modinv(randNum, N)) % N
s,hex(r)

(74550376532497960319987689016107003538808148289652349321148441990401445226647,
 '0x3b78ce563f89a0ed9414f5aa28ad0d96d6795f9c63')

In [56]:
(GenPoint * 86918276961810349294276103416548851884759982251107 ).x % N

96607528561791114397410154718273082025351791959240141029563991069288357310448

In [70]:
t1 = "0x00000000000000000000003b78ce563f89a0ed9414f5aa28ad0d96d6795f9c63"
t1,int(t1,16) < 1 << 184

('0x00000000000000000000003b78ce563f89a0ed9414f5aa28ad0d96d6795f9c63', True)

In [80]:
# s = int("0x6000808080806ea4c4466c164f2cc5b89a3f835803385afa3d82803e3d82f300",16)
# h = int("0x31ab0b330c1bfdce8faea20986848d121c4bddc7de5e7c40382d697dd73f3e05",16)
k = 57896044618658097711785492504343953926418782139537452191302581570759080747168
r = int("0x00000000000000000000003b78ce563f89a0ed9414f5aa28ad0d96d6795f9c63",16)
s = 20424028159692373724824543148434849767490285433474831060123348660748504071752
SK = 1
k,s,r,SK

(57896044618658097711785492504343953926418782139537452191302581570759080747168,
 20424028159692373724824543148434849767490285433474831060123348660748504071752,
 86918276961810349294276103416548851884759982251107,
 1)

In [89]:
# calculating h // hash(msg)
k * s % N - r * SK % N

20620856352409117551791647909372175195204358700435174114179508690798375882738